# About this Jupyter Notebook

@author: Yingding Wang

This notebook demonstrates saving the `log_dir` of tensorboard into a public minio bucket

In [1]:
import sys

# Install kfp to build a pipeline
* Build KF pipeline with SDK: https://www.kubeflow.org/docs/components/pipelines/sdk/build-pipeline/
* Current KFP SDK version on pypi.org: https://pypi.org/project/kfp/ 

In [2]:
!{sys.executable} -m pip install --upgrade --user kfp==1.8.12

In [3]:
!{sys.executable} -m pip show kubernetes

Name: kubernetes
Version: 18.20.0
Summary: Kubernetes python client
Home-page: https://github.com/kubernetes-client/python
Author: Kubernetes
Author-email: 
License: Apache License Version 2.0
Location: /home/jovyan/.local/lib/python3.8/site-packages
Requires: pyyaml, requests, websocket-client, google-auth, python-dateutil, requests-oauthlib, setuptools, six, certifi, urllib3
Required-by: kfp, kfserving


Name: kubernetes
Version: 12.0.1
Summary: Kubernetes python client
Home-page: https://github.com/kubernetes-client/python
Author: Kubernetes
Author-email: 
License: Apache License Version 2.0
Location: /opt/conda/lib/python3.8/site-packages
Requires: pyyaml, requests-oauthlib, setuptools, certifi, six, urllib3, websocket-client, google-auth, requests, python-dateutil
Required-by: kfp, kfserving

## Restart the Kernal

In [4]:
from platform import python_version
print (f"current platform python version: {python_version()}")

current platform python version: 3.8.10


In [5]:
# run kubectl command line to see the quota in the name space
!kubectl describe quota

Name:                                                         kf-resource-quota
Namespace:                                                    kubeflow-kindfor
Resource                                                      Used    Hard
--------                                                      ----    ----
basic-csi.storageclass.storage.k8s.io/persistentvolumeclaims  4       5
basic-csi.storageclass.storage.k8s.io/requests.storage        16Gi    50Gi
cpu                                                           4255m   128
longhorn.storageclass.storage.k8s.io/persistentvolumeclaims   0       10
longhorn.storageclass.storage.k8s.io/requests.storage         0       500Gi
memory                                                        4364Mi  512Gi


## Create K8s secret
* k8s python client API doc: https://github.com/kubernetes-client/python

In [6]:
from kubernetes import client as k8s_client
# from kubernetes import config as k8s_config

K8_NAME_SPACE = 'kubeflow-kindfor'

## Getting started with KubeFlow Python SDK

* Python SDK V2: https://www.kubeflow.org/docs/components/pipelines/sdk-v2/python-function-components/
* Python SDK Overview: https://www.kubeflow.org/docs/components/pipelines/sdk/sdk-overview/

In [7]:
EXPERIMENT_NAME = 'tensorboard'        # Name of the experiment groups runs in the GUI
EXPERIMENT_DESC = 'This example save the log_dir of tensorboard into a public minio bucket'

BASE_PYTHON_IMAGE="tensorflow/tensorflow:2.9.1"
print(f"BASE_PYTHON_IMAGE: {BASE_PYTHON_IMAGE}")

BASE_PYTHON_IMAGE: tensorflow/tensorflow:2.9.1


### V2 Compatible pipeline
* https://www.kubeflow.org/docs/components/pipelines/sdk-v2/v2-compatibility/

In [8]:
import kfp
from kfp.dsl import (
    pipeline,
    ContainerOp
)
from kfp.components import create_component_from_func
from functools import partial

In [9]:
@partial(
    create_component_from_func,
    output_component_file='tf_train_component.yaml',
    base_image=BASE_PYTHON_IMAGE,
    packages_to_install=['tensorflow-datasets==4.5.2'] #tensorflow==2.9.1
)
def tf_training():
    # https://www.tensorflow.org/datasets/keras_example
    import tensorflow as tf
    import tensorflow_datasets as tfds

    (ds_train, ds_test), ds_info = tfds.load(
        'mnist',
        split=['train', 'test'],
        shuffle_files=True,
        as_supervised=True,
        with_info=True,
    )
    def normalize_img(image, label):
        """Normalizes images: `uint8` -> `float32`."""
        return tf.cast(image, tf.float32) / 255., label

    ds_train = ds_train.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
    ds_train = ds_train.cache()
    ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
    ds_train = ds_train.batch(128)
    ds_train = ds_train.prefetch(tf.data.AUTOTUNE)
    
    # Build an evaluation pipeline
    ds_test = ds_test.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
    ds_test = ds_test.batch(128)
    ds_test = ds_test.cache()
    ds_test = ds_test.prefetch(tf.data.AUTOTUNE)
    
    # create and train the model
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10)
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(0.001),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
    )

    model.fit(
        ds_train,
        epochs=6,
        validation_data=ds_test,
    )
    

In [10]:
def pod_resource_transformer(op: ContainerOp, mem_req="200Mi", cpu_req="2000m", mem_lim="2000Mi", cpu_lim='2000m'):
    """
    op.set_memory_limit('1000Mi') = 1GB
    op.set_cpu_limit('1000m') = 1 cpu core
    """
    return op.set_memory_request(mem_req)\
            .set_memory_limit(mem_lim)\
            .set_cpu_request(cpu_req)\
            .set_cpu_limit(cpu_lim)

In [11]:
@pipeline(
    name = EXPERIMENT_NAME,
    description = EXPERIMENT_DESC
)
def minist_train_pipeline():
    train_task = tf_training()
    train_task = pod_resource_transformer(train_task, mem_req="2000Mi", cpu_req="2000m")
    train_task.execution_options.caching_strategy.max_cache_staleness = "P0D"

### Setting imagePullSecretes in K8s with Pipeline config
* Setting imagePullSecretes for Pipeline with SDK: https://github.com/kubeflow/pipelines/issues/5843#issuecomment-859799181

In [12]:
pipeline_config = kfp.dsl.PipelineConf()
pipeline_config.set_image_pull_policy("IfNotPresent")

arguments_nothing = {}

In [13]:
client = kfp.Client()

client.create_run_from_pipeline_func(
    pipeline_func=minist_train_pipeline,
    arguments = arguments_nothing,
    pipeline_conf=pipeline_config,
    experiment_name=EXPERIMENT_NAME,
    namespace=K8_NAME_SPACE,
    # mode=kfp.dsl.PipelineExecutionMode.V1_LEGACY # can not run wiht V2_COMPATIBLE
)

RunPipelineResult(run_id=5cc65aa6-80b8-4bef-b80f-1f990b6196d1)